<a href="https://colab.research.google.com/github/aformen9/BC-ML-Project/blob/agu/notebook/BreastCancer_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para este caso de clasificación de tumores (M vs. B) sobre el dataset de Breast Cancer Wisconsin, te recomendaría emplear Regresión Logística y Random Forest. A continuación, un breve razonamiento de por qué estos dos modelos cumplen bien los requisitos del trabajo:

#Regresión Logística

- Ventajas:

Es un modelo lineal sencillo y muy interpretativo: puedes ver directamente el peso que asigna a cada variable (coeficientes), lo cual ayuda a explicar el resultado en lenguaje comprensible para un público no experto.

Suele servir como “baseline” (modelo base) para cualquier tarea de clasificación binaria; si su desempeño es razonable, sabrás que el problema no es extremadamente complejo.

Su entrenamiento es rápido y, al ser lineal, no requiere tanto ajuste de hiperparámetros.

- Aplicación en este dataset:

Permite ver claramente qué características (por ejemplo, el radio medio, la textura media, el área, etc.) tienden a asociarse más con tumores malignos o benignos.

Al graficar la matriz de confusión, podrás ver cuántos casos de cáncer benigno se clasifican correctamente (TN), cuántos falsos negativos (FN), etc., y explicarlo de forma sencilla: “de cada 100 tumores malignos, el modelo detectó X correctamente y falló en Y”.

#Random Forest

- Ventajas:

Es un ensamble de árboles de decisión que combina múltiples “árboles débiles” para obtener un clasificador robusto, capaz de capturar relaciones no lineales y jerarquías complejas entre variables.

Maneja bien la multicolinealidad y las características con diferentes escalas, y suele dar muy buena precisión en problemas de detección de enfermedades.

Permite obtener la importancia de cada variable (feature importance), lo que facilita justificar por qué ciertas medidas celulares (p. ej., concavidad o suavidad) son más relevantes para predecir malignidad.

- Aplicación en este dataset:

Podrás comparar su desempeño (accuracy, recall, precision, AUC) contra la Regresión Logística. Generalmente Random Forest supera al modelo lineal si hay interacciones o patrones complejos entre las características.

Al graficar la matriz de confusión para Random Forest, mostrarás cuántos tumores malignos detectó correctamente frente a cuántos clasificó erróneamente. Además, puedes incluir un mapa de calor de correlaciones o un boxplot/violin para justificar por qué ciertos grupos de variables (por ejemplo, medidas de perímetro o área) son marcadores fuertes y cómo Random Forest las aprovecha.

#¿Por qué no elijo Decision Tree como segundo modelo?
- Árbol de Decisión simple:

Aunque es muy interpretable y visual (puedes dibujar el árbol completo), tiende a sobreajustarse con facilidad, sobre todo en datasets con muchas variables numéricas y ruido, como este.

Al incluir Random Forest en lugar de un único árbol, te aseguras de “proteger” esa interpretación de árboles simples contra el sobreajuste, ya que el ensamble promedia múltiples árboles.

Sin embargo, podrías usar Decision Tree como experimento adicional (pues el enunciado lo permite), pero para el entregable final te alcanzan dos modelos: uno lineal (Regresión Logística) y uno no lineal/ensemble (Random Forest).

In [2]:
!git clone https://github.com/aformen9/BC-ML-Project.git
%cd BC-ML-Project

Cloning into 'BC-ML-Project'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 20.80 KiB | 10.40 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/BC-ML-Project


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
cancer_data = pd.read_csv('/content/Breast Cancer Wisconsin_TF - Breast Cancer Wisconsin_TF.csv')
cancer_data.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/content/Breast Cancer Wisconsin_TF - Breast Cancer Wisconsin_TF.csv'

In [ ]:
cancer_data.info()

In [ ]:
cancer_data.describe()